<table style="border: none; text-align: center" width="100%">
  <tr style="border: none">
    <td style="min-width:250px; border:none; text-align:left" bgcolor="white">
      <a href="https://www.audiolabs-erlangen.de/" target="_blank">
        <img src="./files/logo_alabs.svg" style="height:30px; display:inline-block" />
      </a>
    </td>
    <td style="min-width:250px; border:none; text-align:center" bgcolor="white">
      <a href="https://www.fau.de/" target="_blank">
        <img src="./files/logo_fau.svg" style="height:50px; display:inline-block" />
      </a>
    </td>
    <td style="min-width:250px; border:none; text-align:right" bgcolor="white">
      <a href="https://www.iis.fraunhofer.de/" target="_blank">
        <img src="./files/logo_iis.svg" style="height:30px; display:inline-block" />
      </a>
    </td>
</tr>
</table>

<div style="text-align: center">
<h1>Harmonic Percussive Source Separation</h1>
<b>Lab Course</b><br/>
<b>International Audio Laboratories Erlangen</b><br/>
<b>Prof. Dr. Meinard Müller</b><br/>
<b>Winter Term 2019/20</b>
</div>

<br/>

<div class="list-group">
  <a style="padding: 7px" class="list-group-item disabled"><b>Authors</b></a>
  <a style="padding: 7px" href="https://www.audiolabs-erlangen.de/fau/professor/mueller" class="list-group-item">Meinard Müller</a>
  <a style="padding: 7px" href="https://www.audiolabs-erlangen.de/fau/assistant/driedger" class="list-group-item">Jonathan Driedger</a>
    <a style="padding: 7px" href="https://www.audiolabs-erlangen.de/fau/assistant/praetzlich" class="list-group-item">Thomas Prätzlich</a>
  <a style="padding: 7px" href="https://www.audiolabs-erlangen.de/fau/assistant/zalkow" class="list-group-item">Frank Zalkow</a>
</div>

<div>
<em>Harmonic Percussive Source Separation</em>, &copy; October 2019
</div>

## <a name="abstract"></a> Abstract

Sounds can broadly be classified into two classes. Harmonic
sound on the one hand side is what we perceive as
pitched sound and what makes us hear melodies and chords.
Percussive sound on the other hand is noise-like and usually
stems from instrument onsets like the hit on a drum or from
consonants in speech. 
The goal of harmonic-percussive
source separation (HPSS) is to decompose an input audio signal
into a signal consisting of all harmonic sounds
and a signal consisting of all percussive sounds.
In this lab course, we study an HPSS algorithm and implement it in Python.
Exploiting knowledge about the spectral structure of harmonic and percussive sounds,
this algorithm decomposes the spectrogram of the given input signal
into two spectrograms, one for the harmonic, and one for the percussive component.
Afterwards, two waveforms are reconstructed from the spectrograms
which finally form the desired signals.
Additionally, we describe the application of HPSS for enhancing chroma feature extraction and onset detection.
The techniques used in this lab cover median filtering, spectral masking and
the inversion of the short-time Fourier transform.

## <a name="hpss"></a> Harmonic-Percussive Source Separation

When listening to our environment, there exists a wide variety of different sounds.
However, on a very coarse level, many sounds can be
categorized to belong in either one of two classes:
harmonic or percussive sounds.
Harmonic sounds are the ones which we perceive to have a certain *pitch* such that we could for example sing along to them.
The sound of a violin is a good example of a harmonic sound.
*Percussive* sounds often stem from two colliding objects like for example the two shells of castanets.
An important characteristic of percussive sounds is that they do not have a pitch but a very clear
localization in time.
Many real-world sounds are mixtures of harmonic and percussive components.
For example, a note played on a piano has a percussive onset (resulting from the hammer hitting the strings)
preceding the harmonic tone (resulting from the vibrating string).

<div class="alert alert-block alert-warning">
<b>Homework Exercise 1</b><br>
Think about three real world examples of sounds which are clearly harmonic and three examples of
      sounds which are clearly percussive.
</div>

Harmonic
- Bell rings
- Singing
- Bee flying

Percussive
- Drum
- Knocking
- Clap

<div class="alert alert-block alert-warning">
What are characteristics of harmonic and percussive signals? Sketch a 
      waveform of a percussive signal and the waveform of a harmonic signal. What are the main
      differences between those waveforms?
</div>

- Harmonic signals has harmonics that are integer multiple of the fundamental frequency, which has a horizontal structure.
- Percussive signals is transient and noise-like, which has a vertical structure.

The goal of harmonic-percussive source separation (HPSS) is to decompose a given input signal into a
sum of two component signals,
one consisting of all harmonic sounds and the other consisting of all percussive sounds.
The core observation in many HPSS algorithms is that in a spectrogram representation of the
input signal, harmonic sounds tend to form horizontal structures (in time-direction), while
percussive sounds form vertical structures (in frequency-direction).
For an example, have a look at following Figure where you can see the power spectrograms of two signals.
The left Figure shows the power spectrogram of a sine-tone with a frequency of
$4000$ Hz and a duration of one second.
This tone is as harmonic as a sound can be.
The power spectrogram shows just one horizontal line.
Contrary, the power spectrogram on the rigtht side shows just one vertical line.
It is the spectrogram of a signal which is zero everywhere, except for the sample at $0.5$ seconds  where it is one.
Therefore, when listening to this signal, we just hear a brief ``click'' at $0.5$ seconds.
This signal is the prototype of a percussive sound.
The same kind of structures can be observed in lower Figure, which shows a
spectrogram of a violin recording and a spectrogram of a castanets recording.

<table>
  <tr>
    <th>
      <img src="./files/figure_spec_sinusoid.svg">
     </th>
     <th>
      <img src="./files/figure_spec_impulse.svg">
     </th>
   </tr>
   <tr>
    <th style="text-align: center;">
      Spectrogram of an ideal harmonic signal.
     </th>
     <th style="text-align: center;">
      Spectrogram of an ideal percussive signal.
     </th>
   </tr>
   <tr>
    <th>
      <img src="./files/figure_spec_violin.svg">
     </th>
     <th>
      <img src="./files/figure_spec_castanets.svg">
     </th>
   </tr>
   <tr>
    <th style="text-align: center;">
      Spectrogram of a recording of a violin.
     </th>
     <th style="text-align: center;">
      Spectrogram of a recording of a castanets.
     </th>
   </tr>
</table>

Real world signals are usually mixtures of harmonic and percussive sounds.
Furthermore, there is no absolute definition of when a sound stops "being harmonic" and starts "being percussive".
Think, for example, of white noise which cannot be assigned to either one of these classes.
However, with the above observations it is possible to decide if a time-frequency instance
of a spectral representation of the input signal, like the short-time Fourier transform (STFT), belongs rather to 
the harmonic component or rather to the percussive component.
This can be done in the following way.
Assume we want to find out if a time-frequency bin in the STFT of the input signal belongs
to the harmonic component.
In this case, the bin should be part of some horizontal, and therefore harmonic structure.
We can check this by first applying some filter to the power spectrogram of the STFT, which enhances horizontal structures
and suppresses vertical structures and see if the filtered bin has some "high value".
However, even if its value is high, it might still belong to some even stronger vertical, and therefore
percussive structure.
We therefore apply another filter to the power spectrogram which enhances vertical structures and suppresses horizontal structures.
Now, in the case that the value of our bin in this vertically enhanced spectrogram is lower than in the horizontally enhanced
spectrogram, it is very likely that it belongs to some harmonic sound and we can assign it to the harmonic component.
Otherwise, if its value was higher in the vertically enhanced spectrogram, we directly know that it is rather
part of some percussive sound and assign it to the percussive component.
This way, we can decide for every time-frequency instance of the original STFT of the input signal whether it belongs 
to the harmonic, or to the percussive component and construct two new STFTs.
In the STFT for the harmonic component, all bins which were assigned to the percussive component are set to zero,
and vice versa for the percussive component.
Finally, by "inverting" these STFTs, we get the audio signals for the harmonic and the percussive component.

   

<div class="alert alert-block alert-warning">
<b>Homework Exercise 2</b><br>
Suppose you apply an HPSS algorithm to white noise. Recall that white noise has a constant power spectral density (it is also said to be flat). What do you expect the harmonic and the percussive
      component to sound like?
</div>

<div class="alert alert-block alert-warning">
If you apply an HPSS algorithm to a recording of your favorite rock band. What do you expect
      the harmonic and the percussive component to sound like?
</div>

## <a name="algoritm"></a> An HPSS Algorithm

We will now describe an actual HPSS algorithm.
Formally, given a discrete input audio signal $x:{\mathbb Z}\to{\mathbb R}$, the algorithm should compute a harmonic component signal $x_\mathrm{h}$
and a percussive component signal $x_\mathrm{p}$, such that $x = x_\mathrm{h} + x_\mathrm{p}$.  
Furthermore, the signals  $x_\mathrm{h}$ and $x_\mathrm{p}$ contain the harmonic and percussive sounds of $x$, respectively.
In the following we describe the consecutive steps of an HPSS algorithm.
We start with the computation of the *STFT* (Subsection [STFT](#STFT)) and proceed
with enhancing the power spectrogram using *median filtering* (Subsection [Median Filtering](#MedFilt)).
Afterwards, the filtered spectrograms are used to compute *binary masks* (Subsection [Binary Masking](#Mask)) which are
used to construct STFTs for the harmonic and the percussive component.
These STFTs are finally transformed back to the time domain (Subsection [ISTFT](#ISTFT)).

### <a name="STFT"></a> STFT

In the first step, we compute  the short-time Fourier transform (STFT) ${\mathcal X}$ of the signal $x$ as:

\begin{equation}
    {\mathcal X}(m,k):= \sum_{n=0}^{N-1} x(n + m H)w(n)\exp(-2\pi ikn/N)
\end{equation}

with ${m\in[0:M-1]:=\{0,\ldots,M -1\}}$ and $k\in[0:N-1]$, where $M$ is the number of frames,
$N$ is the frame size and length of the discrete Fourier transform,
${w:[0:N -1]\to{\mathbb R}}$ is a window function and $H$ is the hopsize.
From ${\mathcal X}$ we can then derive the power spectrogram ${\mathcal Y}$ of $x$:

\begin{equation}
    {\mathcal Y}(m,k) := |{\mathcal X}(m,k)|^2.
\end{equation}

<div class="alert alert-block alert-warning">
<b>Homework Exercise 3</b><br>
The parameters of the STFT have a crucial influence on the HPSS algorithm. Think about what happens to ${\mathcal Y}$ in the case
      you choose $N$ to be very large or very small. How could this influence the algorithm? (Hint: Think about how $N$ influences the time- and frequency-resolution of the STFT.)
</div>

<div class="alert alert-block alert-warning">
Explain in technical terms why harmonic sounds form horizontal and percussive sounds form vertical structures in spectrograms. (Hint: Have a look at the exponential basis functions of the STFT. What does one of these functions describe? How can an impulse be represented with them?)
</div>

<div class="alert alert-block alert-warning">
<b>Lab Experiment 1</b><br>
Load an audio file <samp>CastanetsViolin.wav</samp> using <samp>sf.read</samp>.
</div>

In [ ]:
import soundfile as sf
from IPython.display import Audio

# your code here...

Audio(x, rate=Fs)

<div class="alert alert-block alert-warning">
Compute the STFT ${\mathcal X}$ of the input signal $x$ using <samp>librosa.stft</samp> with the parameters <samp>N=1024</samp>, <samp>H=512</samp>, and a hann-window.
</div>

In [ ]:
import librosa

# your code here...

<div class="alert alert-block alert-warning">
Compute the power spectrogram ${\mathcal Y}$.
</div>

In [ ]:
import numpy as np

# your code here...

<div class="alert alert-block alert-warning">
Visualize ${\mathcal Y}$. Can you spot harmonic and percussive structures?
      Apply logarithmic compression with $\gamma=10$ (see STFT Lab for details on this) when visualizing spectrograms.
</div>

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

# your code here...

<div class="alert alert-block alert-warning">
Do the same for the parameters $N=128$, $H=64$, a hann-window, and $N=8192$, $H=4096$, and a hann-window.
      How do the spectrograms change when you change the parameters? What happens to the harmonic and percussive structures?
</div>

In [ ]:
# your code here...

### <a name="MedFilt"></a> Median Filtering

In the next step, we want to compute a *harmonically enhanced* spectrogram $\tilde{{\mathcal Y}}_\mathrm{h}$ and
a percussively enhanced spectrogram $\tilde{{\mathcal Y}}_\mathrm{p}$ by filtering ${\mathcal Y}$.
This can be done by using a *median filter*.
The median of a list of numbers can be found by arranging all numbers from lowest to highest value and picking
the middle one.
E.g. the median of the list $(7, 3, 4, 6, 5)$ is $5$.
Formally, let $A = (a_1, a_2, \dots, a_L)$ be a list of length $L \in {\mathbb N}$ consisting of real numbers $a_l \in {\mathbb R}, l \in [1:L]$.
First, the elements of $A$ are sorted in ascending order. This results in a list
$\tilde{A} = (\tilde{a}_1, \tilde{a}_2, \dots, \tilde{a}_L)$
with $\tilde{a}_l \leq \tilde{a}_m$ for $l < m$ and $l, m \in [1:L]$.
Then, the ${\mathrm{median}}$ of $A$ is defined as

\begin{equation}
{\mathrm{median}}(A) := \begin{cases} \tilde{a}_{(L+1)/2} & \mbox{for $L$ being odd}\\ (\tilde{a}_{L/2} + \tilde{a}_{L/2+1})/2 & \mbox{otherwise}\end{cases}
\end{equation}

Now, given a matrix $B\in{\mathbb R}^{M\times K}$, we define harmonic and percussive median filters

\begin{eqnarray}
    {\mathrm{medfilt}_\mathrm{h}}(B)(m,k) := {\mathrm{median}}(\{B(m-\ell_\mathrm{h},k),\ldots,B(m+\ell_\mathrm{h},k)\})\\
    {\mathrm{medfilt}_\mathrm{p}}(B)(m,k) := {\mathrm{median}}(\{B(m,k-\ell_\mathrm{p}),\ldots,B(m,k+\ell_\mathrm{p})\})
\end{eqnarray}

for $M,K,\ell_\mathrm{h},\ell_\mathrm{p}\in{\mathbb N}$, where $2\ell_\mathrm{h} + 1$ and $2\ell_\mathrm{p} + 1$ are the lengths of the median filters, respectively.
Note that we simply assume $B(m,k)=0$ for $m \notin [0:M-1]$ or $k \notin [0:K-1]$.
The enhanced spectrograms are then computed as

\begin{eqnarray}
    \tilde{{\mathcal Y}}_\mathrm{h} := {\mathrm{medfilt}_\mathrm{h}}({\mathcal Y})\\
    \tilde{{\mathcal Y}}_\mathrm{p} := {\mathrm{medfilt}_\mathrm{p}}({\mathcal Y})
\end{eqnarray}

<div class="alert alert-block alert-warning">
<b>Homework Excercise 4</b><br>
The arithmetic ${\mathrm{mean}}$ of a set $A\subset{\mathbb R}$ of size $N$ is defined as ${{\mathrm{mean}}(A):= \frac{1}{N}\sum_{n=0}^{N-1}a_n}$.
      Compute the ${\mathrm{median}}$ and the ${\mathrm{mean}}$ for the set ${A=\{2, 3, 190, 2, 3\}}$.
      Why do you think the HPSS algorithm employs median filtering and not mean filtering?
</div>

<div class="alert alert-block alert-warning">
Apply a horizontal and a vertical median filter of length $3$ to the matrix

\begin{equation*}
B =
\begin{bmatrix}
1 & 1 & 46 & 2 \\
3 & 1 & 50 & 1 \\
60 & 68 & 70 & 67 \\
2 & 1 & 65 & 1
\end{bmatrix}
\end{equation*}
</div>

In [ ]:
from scipy import signal
import numpy as np

# your code here...

B = np.array([[1, 1, 46, 2], [3, 1, 50, 1], [60, 68, 70, 67], [2, 1, 65, 1]], dtype='float64')
print('Origignal B:')
print(B)
print('Horizontally filtered B:')
print(horizontal_median_filter(B, 3))
print('Vertically filtered B:')
print(vertical_median_filter(B, 3))

<div class="alert alert-block alert-warning">
Explain in your own words why median filtering allows for enhancing/suppressing harmonic/percussive structures in a spectrogram.
</div>

<div class="alert alert-block alert-warning">
<b>Lab Experiment 2</b><br>
Apply harmonic and percussive median filters to the power spectrogram ${\mathcal Y}$ which you computed in the previous exercise (<samp>N=1024</samp>, <samp>H=512</samp>, and a hann-window using <samp>scipy.signal.medfilt</samp>.
</div>

In [ ]:
# your code here...

<div class="alert alert-block alert-warning">
Play around with different filter lengths (3, 11, 51, 101). Visualize the filtered spectrograms. What are your observations?
</div>

In [ ]:
# your code here...

### <a name="Mask"></a>  Binary Masking

Having the enhanced spectrograms $\tilde{{\mathcal Y}}_\mathrm{h}$ and $\tilde{{\mathcal Y}}_\mathrm{p}$, we now need to assign all
time-frequency bins of ${\mathcal X}$ to either the harmonic or the percussive component.
This can be done by *binary masking*.
A binary mask is a matrix ${\mathcal M}\in\{0,1\}^{M\times K}$.
It can be applied to an STFT ${\mathcal X}$ by computing
${\mathcal X} \odot {\mathcal M}$, where the operator $\odot$ denotes point-wise multiplication.
A mask value of one preserves the value in the STFT and a mask value of zero suppresses it.
For our HPSS algorithm, the binary masks are defined by comparing the values in the enhanced
spectrograms $\tilde{{\mathcal Y}}_\mathrm{h}$ and $\tilde{{\mathcal Y}}_\mathrm{p}$.

\begin{eqnarray}
{\mathcal M}_\mathrm{h}(m,k) :=
\begin{cases}
   1 & \text{if } \tilde{{\mathcal Y}}_\mathrm{h}(m,k) \geq \tilde{{\mathcal Y}}_\mathrm{p}(m,k) \\
   0       & \text{else}
  \end{cases} \\
{\mathcal M}_\mathrm{p}(m,k) :=
\begin{cases}
   1 & \text{if } \tilde{{\mathcal Y}}_\mathrm{p}(m,k) > \tilde{{\mathcal Y}}_\mathrm{h}(m,k) \\
   0       & \text{else.}
  \end{cases}
\end{eqnarray}

Applying these masks to the original STFT ${\mathcal X}$ yields the STFTs for the harmonic and the percussive component of the signal
${{\mathcal X}_\mathrm{h} := ({\mathcal X} \odot {\mathcal M}_\mathrm{h})}$ and ${{\mathcal X}_\mathrm{p} := ({\mathcal X} \odot {\mathcal M}_\mathrm{p})}$.
Note that by the definition of ${\mathcal M}_\mathrm{h}$ and ${\mathcal M}_\mathrm{p}$, it holds that ${\mathcal M}_\mathrm{h}(m,k)+{\mathcal M}_\mathrm{p}(m,k) = 1$
for $m \in[0:M-1]$, $k\in[0:K-1]$.
Therefore, every time-frequency bin of ${\mathcal X}$ is assigned either to ${\mathcal X}_\mathrm{h}$ or ${\mathcal X}_\mathrm{p}$.

<div class="alert alert-block alert-warning">
<b>Homework Excercise 5</b><br>
Assume you have the two enhanced spectrograms

  \begin{equation*}
  \begin{array}{cc}
      \tilde{{\mathcal Y}}_\mathrm{h} =
      \begin{bmatrix}
      1 & 1 & 2 & 2 \\
      1 & 3 & 1 & 1 \\
      60 & 68 & 68 & 67 \\
      1 & 2 & 1 & 1
      \end{bmatrix}, &
      \tilde{{\mathcal Y}}_\mathrm{p} =
      \begin{bmatrix}
      1 & 1 & 46 & 1 \\
      3 & 1 & 50 & 2 \\
      2 & 1 & 65 & 1 \\
      2 & 1 & 65 & 1
      \end{bmatrix}
  \end{array}
  \end{equation*}

Compute the binary masks ${\mathcal M}_\mathrm{h}$ and ${\mathcal M}_\mathrm{p}$ and apply them to the matrix
      \begin{equation*}
      {\mathcal X} =
      \begin{bmatrix}
      1 & 1 & 46 & 2 \\
      3 & 1 & 50 & 1 \\
      60 & 68 & 70 & 67 \\
      2 & 1 & 65 & 1
      \end{bmatrix}
      \end{equation*}
</div>

In [ ]:
Y_h_toy = np.array([[1 ,1, 2, 2], [1, 3, 1, 1], [60, 68, 68, 67], [1, 2, 1, 1]])
Y_p_toy = np.array([[1, 1, 46, 1], [3, 1, 50, 2], [2, 1, 65, 1], [2, 1, 65, 1]])
X_toy = np.array([[1, 1, 46, 2], [3, 1, 50, 1], [60, 68, 70, 67], [2, 1, 65, 1]])

# your code here...

<div class="alert alert-block alert-warning">
<b>Lab Experiment 3</b><br>
Use the median filtered power spectrograms $\tilde{{\mathcal Y}}_\mathrm{h}$ and $\tilde{{\mathcal Y}}_\mathrm{p}$ from the previous exercise (filter length 11) to compute the binary masks ${\mathcal M}_\mathrm{h}$ and ${\mathcal M}_\mathrm{p}$.
</div>

In [ ]:
# your code here...

<div class="alert alert-block alert-warning">
Visualize the masks (this time without logarithmic compression).
</div>

In [ ]:
# your code here...

<div class="alert alert-block alert-warning">
Apply the masks to the original STFT ${\mathcal X}$ to compute ${\mathcal X}_\mathrm{h}$ and ${\mathcal X}_\mathrm{p}$.
</div>

In [ ]:
# your code here...

<div class="alert alert-block alert-warning">
Visualize the power spectrograms ${\mathcal Y}_\mathrm{h}$ and ${\mathcal Y}_\mathrm{p}$ of ${\mathcal X}_\mathrm{h}$ and ${\mathcal X}_\mathrm{p}$.
</div>

In [ ]:
# your code here...

### <a name="ISTFT"></a>  Inversion of the Short-Time Fourier Transform

In the final step, we need to transform our constructed STFTs ${\mathcal X}_\mathrm{h}$ and ${\mathcal X}_\mathrm{p}$ back to the time-domain.
To this end, we apply an "inverse" STFT to these matrices to compute the component signals $x_\mathrm{h}$ and $x_\mathrm{p}$.
Note that the topic "inversion of the STFT" is not as trivial as it might seem at the first glance.
In the case that ${\mathcal X}$ is the original STFT of an audio signal $x$, and further preconditions are satisfied (for example that $N \geq H$ for $N$ being the size of the discrete Fourier transform and $H$ being the hopsize of the STFT),
it is possible to invert the STFT and to reconstruct $x$ from ${\mathcal X}$ perfectly.
However, as soon as the original STFT ${\mathcal X}$ has been modified to some $\tilde{{\mathcal X}}$, for example by masking, there might be no audio signal which has exactly $\tilde{{\mathcal X}}$ as its STFT.
In such a case, one usually aims to find an audio signal whose STFT is "approximately" $\tilde{{\mathcal X}}$.
For this Lab Course, you can simply assume that you can invert the STFT using <samp>librosa.istft</samp>.

<img src="./files/HPSep3.png">

<div class="alert alert-block alert-warning">
<b>Homework Excercise 6</b><br>
Assume ${\mathcal X}$ is the original STFT of some audio signal $x$. Why do we need the precondition $N \geq H$ for $N$ being the size of the discrete Fourier transform and $H$ being the hopsize of the STFT to reconstruct $x$ from ${\mathcal X}$ perfectly?
</div>

<div class="alert alert-block alert-warning">
<b>Lab Experiment 4</b><br>
Apply the inverse STFT function <samp>librosa.istft</samp> to $X_\mathrm{h}$ and $X_\mathrm{p}$ from the previous experiment and listen to the results.
</div>

In [ ]:
from IPython.display import display

# your code here...

display(Audio(x_h, rate=Fs))
display(Audio(x_p, rate=Fs))

<div class="alert alert-block alert-warning">
Save the computed harmonic and percussive component by using <samp>sf.write</samp>.
</div>

In [ ]:
from IPython.display import FileLink

# your code here...

display(FileLink('x_h.wav'))
display(FileLink('x_p.wav'))

### <a name="intepret"></a>  Physical Interpretation of Parameters

Note that one can specify the filter lengths of the harmonic and percussive median filters  in seconds and Hertz, respectively.
This  makes their physical interpretation easier.
Given the sampling rate $F_\mathrm{s}$ of the input signal $x$ as well as the frame length $N$ and the hopsize $H$,
we can convert filter lengths given in seconds and Hertz to filter lengths given in indices

\begin{eqnarray}
L_\mathrm{h}(t):=\left\lceil \frac{F_\mathrm{s}}{H} t \right\rceil\label{eqn:L_h}\\
L_\mathrm{p}(d):=\left\lceil \frac{N}{F_\mathrm{s}} d \right\rceil\label{eqn:L_p}
\end{eqnarray}

<div class="alert alert-block alert-warning">
<b>Homework Excercise 7</b><br>
Assume $F_\mathrm{s}=22050$ Hz, $N=1024$, and $H=256$. Compute $L_\mathrm{h}(0.5\text{ sec})$ and $L_\mathrm{p}(600 \text{ Hz})$.
</div>

In [ ]:
# your code here...

<div class="alert alert-block alert-warning">
<b>Lab Experiment 5</b><br>
Complete the implementation of the HPSS algorithm in the function <samp>HPSS</samp>:

<ol>
<li> Compute the STFT ${\mathcal X}$ of the input signal $x$ using <samp>librosa.stft </samp>.</li>
<li> Compute the power spectrogram ${\mathcal Y}$ from the ${\mathcal X}$.</li>
<li> Convert the median filter lengths from seconds and Hertz to indices. if a filter length is even, subtract 1.</li>
<li> Apply median filters to ${\mathcal Y}$ using <samp>scipy.signal.medfilt</samp> to compute ${\mathcal Y}_\mathrm{h}$ and ${\mathcal Y}_\mathrm{p}$.</li>
<li> Derive the masks ${\mathcal M}_\mathrm{h}$ and ${\mathcal M}_\mathrm{p}$ from ${\mathcal Y}_\mathrm{h}$ and ${\mathcal Y}_\mathrm{p}$.</li>
<li> Compute ${\mathcal X}_\mathrm{h}$ and ${\mathcal X}_\mathrm{p}$.</li>
<li> Apply the inverse STFT (<samp>librosa.istft </samp>) to get $x_\mathrm{h}$ and $x_\mathrm{p}$.</li>
</ol>
</div>

In [ ]:
def HPSS(x, N, H, w, Fs, lh_sec, lp_Hz):
    # x:      Input signal
    # N:      Frame length
    # H:      Hopsize
    # w:      Window function of length N
    # Fs:     Sampling rate of x
    # lh_sec: Horizontal median filter length given in seconds
    # lp_Hz:  Percussive median filter length given in Hertz

    # stft

    # power spectrogram

    # median filtering

    # masking

    # istft

    return x_h, x_p

<div class="alert alert-block alert-warning">
Test your implementation:

<ol>
<li> Load the audio files <samp>Stepdad.wav</samp>, <samp>Applause.wav</samp>, and <samp>DrumSolo.wav</samp> from the <samp>Data</samp> folder.</li>
<li> Apply HPSS using the parameters <samp>N=1024</samp>, <samp>H=512</samp>, a hann-window, <samp>lh_sec=0.2</samp>, and <samp>lp_Hz=500</samp> to all loaded signals.</li>
<li>Listen to the results.</li>
</ol>
</div>

In [ ]:
for file in ('files/Stepdad.wav', 'files/Applause.wav', 'files/DrumSolo.wav'):
    print('# ' + file)
    
    # your code here...
    
    print('Orignal')
    display(Audio(x, rate=Fs))
    print('Harmonic Component')
    display(Audio(x_h, rate=Fs))
    print('Percussive Component')
    display(Audio(x_p, rate=Fs))
    print('')

## <a name="application"></a> Applications of HPSS

In many audio processing tasks, the essential information lies in either the harmonic or the percussive component of an audio signal.
In such cases, HPSS is very well suited as a pre-processing step to enhance the outcome of an algorithm.
In the following, we introduce two procedures that can be improved by applying HPSS.
The harmonic component from the HPSS algorithm can be used to enhance chroma features (Subsection [Chroma](#chroma)) and the  percussive component helps to improve the results of an onset detection procedure (Subsection [Onset](#onset)).

## <a name="chroma"></a> Enhancing Chroma Features using HPSS

Two pitches sound similar when they are an octave apart from each other (12 tones in the equal tempered scale).
We say that these pitches share the same chroma which we refer to by the pitch spelling names $\{\mathrm{C},\mathrm{C}^{\sharp},\mathrm{D},\mathrm{D}^{\sharp},\mathrm{E},\mathrm{F},\mathrm{F}^{\sharp},\mathrm{G},\mathrm{G}^{\sharp},\mathrm{A},\mathrm{A}^{\sharp},\mathrm{B}\}$.
Chroma features exploit the above observation, by adding up all frequency bands in a power spectrogram that belong to the same chroma.
Technically this can be realized by the following procedure.
First we assign a pitch index (MIDI pitch number) to each frequency index $k \in [1:N/2-1]$ of the spectrogram by using the formula:

\begin{equation}
    p(k) =  \text{round}\left(12\log_2\left( \frac{ k\cdot F_\mathrm{s}}{440 \cdot N}\right)\right)+69.
\end{equation}

where $N$ is the number of frequency bins in the spectrogram and $F_\mathrm{s}$ is the sampling rate of the audio signal.
Note that $p$ maps frequency indices corresponding to frequencies around the chamber tone A4 (440 Hz) to its MIDI pitch number 69.
Then we add up all frequency bands in the power spectrogram belonging to the same chroma $c \in [0:11]$:

\begin{equation}
      {\mathcal C}(m,c) := \sum_ {\{k |\,p(k)\,\mathrm{mod}\,12 = c\,\}}{{\mathcal Y}(m,k)}
\end{equation}

where $m\in [0:M-1]$ and $M$ is the number of frames.

Chroma features are correlated with the pitches and the harmonic structure of music.
Pitches usually form horizontal structures in the spectrogram, whereas transient or percussive sounds form vertical structures. Percussive sounds have a negative impact on the chroma extraction, as they "activate all frequencies" in the spectrogram.
Hence, one way to improve the chroma extraction is to first apply HPSS and to perform the chroma extraction on the power spectrogram of the harmonic component signal ${\mathcal Y}_\mathrm{h}(m,k) = |{\mathcal X_\mathrm{h}(m,k)}|^2$.

<div class="alert alert-block alert-warning">
<b>Lab Experiment 6</b><br>
Apply the HPSS algorithm as a pre-processing step in a chroma extraction procedure:

<ol>
<li> Load the file <samp>CastanetsViolin.wav</samp>.</li>
<li> Compute chroma features on $x$ with the parameters <samp>N=4410</samp> and <samp>H=2205</samp>.</li>
<li> Visualize the chroma features.</li>
<li> Apply your HPSS algorithm to separate the castanets from the violin.</li>
<li> Use the harmonically enhanced signal $x_\mathrm{h}$ to compute chroma features and visualize them.</li>
<li> Now compare the visualization of the chroma extracted from the original signal $x$ and the chroma extracted from the harmonic component signal $x_\mathrm{h}$. What do you observe?</li>
</ol>


</div>

In [ ]:
from numpy.linalg import norm

def simple_chroma(x, N, H, Fs):
    # x:       input signal
    # N:       frame length
    # H:       hopsize
    # Fs:      sampling rate
    
    X = librosa.stft(x, N, H, N, window='hann', pad_mode='constant')
    Y = np.abs(X) ** 2
    Y_comp = np.log(1 + 0.5 * Y)
    C = np.zeros((12, Y.shape[1]), dtype=Y.dtype)
    n = np.arange(1, Y.shape[0])
    pitches = np.round(12 * np.log2((Fs * n/N) / 440)) + 69
    chroma_mapping = pitches % 12
    chroma_mapping = np.insert(chroma_mapping, 0, -1) # never use DC offset
    for k in range(12):
        C[k, :] = Y_comp[chroma_mapping == k, :].sum(axis=0)
    C = normalize_chroma(C, 2, 0.0001)
        
    return C

def normalize_chroma(C, norm_p, threshold):
    f_norm = norm(C, norm_p, axis=0)
    C_norm = C.copy()
    C_norm[:, f_norm >= threshold] /= f_norm
    return C_norm

# your code here...

## <a name="onset"></a> HPSS for Onset Detection

Onset detection is the task of finding the temporal positions of note onsets in a music recording.
More concrete, the task could be to detect all time positions on which some drum is hit
in a recording of a rock song.
One way to approach this problem is to assume, that drum hits emit a short burst of high energy
and the goal is therefore to detect these bursts in the input signal.
To this end, one first computes the *short-time power* ${\mathcal P}$ of the input signal $x$ by

\begin{equation}
    {\mathcal P}(m):= \sum_{n=0}^{N-1} x(n + m H)^2 \quad
\end{equation}

where $H$ is the hopsize and $N$ is the length of one frame (similar to the computation of the STFT).
Since we are looking for time-positions of high energy, the goal is therefore to detect peaks in ${\mathcal P}$.
A common technique to enhance peaks in a sequence is to
subtract the *local average* $\tilde{{\mathcal P}}$ from ${\mathcal P}$ itself. $\tilde{{\mathcal P}}$ is defined by

\begin{equation}
\tilde{{\mathcal P}}(m) := \sum_{j=-J}^{J}{\mathcal P}(m+j) \frac{1}{2J+1}
\end{equation}

for a neighborhood $J\in{\mathbb N}$, $m\in[0:M-1]$, and $M$ is the number of frames.
Note that we assume ${\mathcal P}(m) = 0$ for $m\notin [0:M-1]$.
From this, we compute a *novelty curve* ${\mathcal N}$

\begin{equation}
{\mathcal N}(m) := \max(0,{\mathcal P}(m) - \tilde{{\mathcal P}}(m))
\end{equation}

The peaks in ${\mathcal N}$ indicate positions of high energy in $x$, and are therefore potential time positions of drum hits.

This procedure works well in case the initial assumption, namely that onsets or drum hits emit some
burst of energy which stand out from the remaining energy in the signal, is met.
However, especially in professionally mixed music recordings, the short-time energy is often adjusted
to be more or less constant over time (compression).
One possibility to circumvent this problem is to apply HPSS to the input signal prior to the onset detection.
The onset detection is then executed solely on the percussive component which usually contains
all drum hits and satisfies the assumption of having energy bursts at the respective time-positions.

<div class="alert alert-block alert-warning">
<b>Lab Experiment 7</b><br>
Complete the implementation of the onset detection algorithm in the function <samp>onset_detection</samp>:

<ol>
<li> Compute the short-time power ${\mathcal P}$ of the input signal $x$ using the provided function <samp>stp</samp>.</li>
<li> Compute the local average $\tilde{{\mathcal P}}$.
              (Hint: Note that the equation can be formulated as a convolution and that you can compute convolutions in Python using the function <samp>np.convolve</samp>.
              Note further that this command has an keyword <samp>mode</samp>. Finally, have a look at the function <samp>np.ones</samp>.)</li>
<li> Compute the novelty curve ${\mathcal N}$.</li>
</ol>
</div>

In [ ]:
def stp(x, N, H):
    # x:  Input signal
    # N:  Frame length
    # H:  Hopsize
    x_pad = np.concatenate((x, np.zeros(N)))
    num_windows = np.ceil(1 + (len(x) - N) / H)
    win_pos = np.arange(num_windows) * H
    idx = np.array([np.arange(w, w+N) for w in win_pos], dtype='int32')
    P = (x_pad[idx] ** 2).sum(axis=1) / N
    return P
    
def onset_detection(x, N, H, J):
    # x:      Input signal
    # N:      Frame length
    # H:      Hopsize
    # J:      Neighborhood
    
    # short-time power

    # local average

    # novelty
    
    return N

<div class="alert alert-block alert-warning">
Test your implementation by applying it to the audio file <samp>StillPluto_BitterPill.wav</samp>. As a starting point, use $N=882$, $H = 441$, and $J = 10$.
</div>

In [ ]:
# your code here...

plt.figure(figsize=(16, 10.66))
plt.plot(t, novelty)

<div class="alert alert-block alert-warning">
Sonify your results using the function <samp>sonify_noveltyCurve</samp>. This function will generate a stereo audio signal in which you can hear the provided original signal
      in one of the channels. In the other channel, each peak in the provided novelty curve is audible as a click sound. You can therefore check by listening whether the peaks in your
      computed novelty curve are aligned with drum hits in the original signal. To apply the function <samp>sonify_noveltyCurve</samp>, you need to specify the sampling frequency of the
      novelty curve. How can you compute it? (Hint: It is dependent on $H$ and the sampling frequency $F_\mathrm{s}$ of the input audio signal).
</div>

In [ ]:
def sonify_noveltyCurve(novelty, x, Fs, sampling_frequency_novelty):
    pos = np.append(novelty, novelty[-1]) > np.insert(novelty, 0, novelty[0])
    neg = np.logical_not(pos)
    peaks = np.where(np.logical_and(pos[:-1], neg[1:]))[0]
    
    values = novelty[peaks]
    values /= np.max(values)
    peaks = peaks[values >= 0.01]
    values = values[values >= 0.01]
    peaks_idx = np.int32(np.round(peaks / sampling_frequency_novelty * Fs))
    
    sine_periods = 8
    sine_freq = 880
    click = np.sin(np.linspace(0, sine_periods * 2 * np.pi, sine_periods * Fs//sine_freq))
    ramp = np.linspace(1, 1/len(click), len(click)) ** 2
    click = click * ramp
    click = (click * np.abs(np.max(x)))
    
    out = np.zeros(len(x), dtype=x.dtype)
    for i, start in enumerate(peaks_idx):
        idx = np.arange(start, start+len(click))
        out[idx] += (click * values[i]).astype(x.dtype)
    
    return np.vstack((x, out))              

# your code here...

Audio(newx, rate=Fs)

<div class="alert alert-block alert-warning">
Listen to the generated results. What is your impression?
</div>

<div class="alert alert-block alert-warning">
Now apply your HPSS algorithm to the audio file and rerun the detection algorithm on just the percussive component $x_\mathrm{p}$. Again, sonify the results. What is your impression now?
</div>

In [ ]:
# your code here...

plt.figure(figsize=(16, 10.66))
plt.plot(t, novelty)

newx = sonify_noveltyCurve(novelty, x, Fs, Fs/H)
Audio(newx, rate=Fs)

<h4>Reference</h4>
<br/>

<div>
  <a href="http://www.springer.com/gp/book/9783319219448">
    <img src="./files/FMP_Cover.png" alt="FMP_Cover" width="80"  style="float:right;">
  </a>
</div>

<div style="float:left;">
<a href="https://www.audiolabs-erlangen.de/fau/professor/mueller">Meinard Müller</a> <br /> 
Fundamentals of Music Processing &ndash; Audio, Analysis, Algorithms, Applications  <br />
ISBN: 978-3-319-21944-8<br />
483 p., <a href="http://www.springer.com/gp/book/9783319219448">Springer</a>, 2015  <br /> 
<a href="http://www.music-processing.de">www.music-processing.de</a>    <br /> 
<a href="https://www.audiolabs-erlangen.de/fau/professor/mueller/bookFMP">Accompanying Website</a>   
</div>

<h4>Acknowledgment</h4>
<br/>
The International Audio Laboratories Erlangen are a joint institution of the Friedrich-Alexander-Universität Erlangen-Nürnberg (FAU) and Fraunhofer Institute for Integrated Circuits IIS. 